In [4]:
import tensorflow as tf
from pydub import AudioSegment
from pydub.playback import play
import os
import pandas as pd
import numpy as np
import glob
import subprocess
import string
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import librosa

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
loaded_model = tf.keras.models.load_model("C:/Projects/capstone-project/backend/global_emotion_model.h5")
audio_directory = "C:/Projects/capstone-project/backend/uploads"


features_list = []
for filename in os.listdir(audio_directory):
    if filename.endswith(('.wav', '.mp3', '.flac', '.ogg')):
        # load the audio file
        audio_file, sr = librosa.load(os.path.join(audio_directory, filename), sr=None)

        # extract the features
        mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=30)
        chroma = librosa.feature.chroma_stft(y=audio_file, sr=sr, n_chroma=20)

        # concatenate the features into a single feature vector
        features = np.concatenate((mfccs.mean(axis=1), mfccs.var(axis=1), chroma.mean(axis=1)))

        features_list.append(features)
        if not os.path.exists("C:/Projects/capstone-project/backend/acoustic_features"):
            os.mkdir("C:/Projects/capstone-project/backend/acoustic_features")

        acoustic_features_path = "C:/Projects/capstone-project/backend/acoustic_features"


        csv_path = os.path.join(acoustic_features_path, f"{filename}.csv")
        np.savetxt(csv_path, features, delimiter=",")

In [11]:
input_directory = 'C:/Projects/capstone-project/backend/uploads'
output_directory = 'C:/Projects/capstone-project/backend/transcriptions'
linguistic_features_path = "C:/Projects/capstone-project/backend/linguistic_features"



word2vec_model = Word2Vec.load("C:/Projects/capstone-project/backend/word2vec/model.model")

def get_word2vec_vectors(text, model):
    tokens = word_tokenize(text)
    vectors = [model.wv[word] for word in tokens if word in model.wv.key_to_index]
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(model.vector_size)

# Iterate through transcript directories

for filename in os.listdir(output_directory):
        if filename.endswith(".txt"):
            # Initialize lists to store features and emotions for each segment
            linguistic_features = []
            # Read the transcript
            transcript_path = os.path.join(output_directory, filename)
            with open(transcript_path, 'r') as file:
                transcript_text = file.read()

            # Get Word2Vec vectors
            word2vec_vector = get_word2vec_vectors(transcript_text, word2vec_model)

            # Extract emotion from the corresponding acoustic features CSV
            acoustic_csv_path = os.path.join(acoustic_features_path, filename.replace(".txt", ".mp3.csv"))
            if os.path.exists(acoustic_csv_path):
                acoustic_df = pd.read_csv(acoustic_csv_path)
                emotion_label = acoustic_df.iloc[1, 1]  # Assuming emotion is in the second column after the second row
            else:
                emotion_label = "Unknown"  # Handle missing emotion label

            # Append features and emotions to lists
            for feature in word2vec_vector:
                linguistic_features.append(feature)
                emotions.append(emotion_label)

            # Create a DataFrame for linguistic features and emotions
            linguistic_df = pd.DataFrame({
                "Feature": linguistic_features
            })

            # Save the DataFrame to a CSV file in the linguistic features directory
            directory = os.path.join(linguistic_features_path)
            os.makedirs(directory, exist_ok=True)
            output_csv_path = os.path.join(directory, f"{filename.replace('.txt', '.csv')}")
            linguistic_df.to_csv(output_csv_path, index=False)

In [12]:
combined_features_directory = "C:/Projects/capstone-project/backend/combined-features"
if not os.path.exists(combined_features_directory):
  os.makedirs(combined_features_directory)

# Loop through the acoustic features files
for acoustic_file in os.listdir(acoustic_features_path):
  # Get the segment file name
  segment_file_name = acoustic_file.split(".")[0]
  # Find the corresponding linguistic features file
  linguistic_file = segment_file_name + ".csv"
  # Read the acoustic and linguistic features dataframes
  acoustic_df = pd.read_csv(os.path.join(acoustic_features_path, acoustic_file))
  linguistic_df = pd.read_csv(os.path.join(linguistic_features_path, linguistic_file))
  # Rename the first column of the acoustic dataframe to "Feature"
  acoustic_df.rename(columns={acoustic_df.columns[0]: "Feature"}, inplace=True)
  # Remove the header row from the linguistic dataframe
  linguistic_df = linguistic_df.iloc[1:]
  # Rename the columns of the linguistic dataframe to match the acoustic dataframe
  linguistic_df.columns = acoustic_df.columns
  # Append the linguistic features to the acoustic features by rows
  combined_df = pd.concat([acoustic_df, linguistic_df], axis=0)
  # Save the combined dataframe to a new file
  combined_file = segment_file_name + ".csv"
  combined_df.to_csv(os.path.join(combined_features_directory, combined_file), index=False)

In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

emotion_mapping = {
    0: "Happy",
    1: "Sad",
    2: "Confused",
    3: "Other"  # Add more emotions if needed
}


# Load the saved global model
loaded_model = tf.keras.models.load_model("C:/Projects/capstone-project/backend/global_emotion_model.h5")



combined_features_directory = "C:/Projects/capstone-project/backend/combined-features"

feature_files = [os.path.join(combined_features_directory, file) for file in os.listdir(combined_features_directory) if file.endswith(".csv")]

emotions = []

for feature_file in feature_files:
    # Read the CSV file using Pandas
    df = pd.read_csv(feature_file, header=None)

    # Extract features (excluding the first row)
    features = df.iloc[1:, 0].astype(np.float32).values

    # Perform additional preprocessing if needed

    # Predict the emotion using the loaded model
    predicted_emotion = loaded_model.predict(features.reshape(1, -1))  # Reshape to match model input shape

    predicted_emotion_label = emotion_mapping[np.argmax(predicted_emotion)]
    # Map the predicted output to an emotion label (e.g., "happy," "sad," "confused")

    # Append the emotion to the list of emotions
    emotions.append(predicted_emotion_label)

1/1 [==============================] - 0s 40ms/step


In [7]:
emotions

['Confused', 'Confused', 'Confused']